<a href="https://colab.research.google.com/github/zonezero2604/Hkr_Set/blob/master/Bai17_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Giới thiệu về Keras Tuner

## Tổng quát

Keras Tuner là một thư viện giúp bạn chọn bộ siêu tham số (hyperparameters) tối ưu cho chương trình TensorFlow của mình. Quá trình lựa chọn quyền thiết lập các siêu tham số cho  ứng dụng machine learning (ML) được gọi *hyperparameter tuning* hoặc *hypertuning*. 

Hyperparameters là các biến chi phối quá trình đào tạo và cấu trúc liên kết của một mô hình ML. Các biến này không đổi trong quá trình đào tạo và ảnh hưởng trực tiếp đến hiệu suất của chương trình ML của bạn. Siêu tham số có hai loại:
 
1. **Model hyperparameters**  của mô hình ảnh hưởng đến việc lựa chọn mô hình chẳng hạn như số lượng và chiều rộng của các lớp ẩn
2. **Algorithm hyperparameters** ảnh hưởng đến tốc độ và chất lượng của thuật toán học tập, chẳng hạn như tốc độ học cho Stochastic Gradient Descent (SGD) và số lượng lân cận gần nhất cho ak Nearest Neighbors (KNN)

Trong hướng dẫn này, bạn sẽ sử dụng Keras Tuner để thực hiện hypertuning cho ứng dụng phân loại hình ảnh.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

Cài đặt và import the Keras Tuner.

In [2]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 5.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=b2653b37d2ea6efa4137313f05d30ff143e66ea41a7c1b62cc5c4e33de090fd4
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=080fa8499c1bb88941a66109cf07b7c92d10b18a5a4448a2589cc06f65630193
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Download và chuẩn bị tập dữ liệu

Trong hướng dẫn này, bạn sẽ sử dụng Keras Tuner để tìm các siêu tham số tốt nhất cho mô hình máy học phân loại hình ảnh quần áo từ [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). 

Load the data.

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

Khi bạn xây dựng một mô hình cho hypertuning, bạn cũng xác định không gian tìm kiếm hyperparameter ngoài kiến trúc mô hình. Mô hình bạn thiết lập để tăng cường được gọi là   *hypermodel*.

Bạn có thể định nghĩa hypermodel thông qua hai cách tiếp cận:

* Bằng cách sử dụng chức năng xây dựng mô hình (model builder function)
* Bằng cách phân lớp lớp `HyperModel` của Keras Tuner API

Bạn cũng có thể sử dụng hai lớp `HyperModel` - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) và [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) cho các ứng dụng thị giác máy tính.

Trong hướng dẫn này, bạn sử dụng chức năng xây dựng mô hình để định nghĩa mô hình phân loại hình ảnh. Hàm tạo mô hình trả về một mô hình đã biên dịch và sử dụng các hyperparameters mà bạn định nghĩa để hypertune model.
 

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value = 32, max_value = 512, step = 32)
  hp_units2 = hp.Int('units2', min_value = 32, max_value = 512, step = 32)
  hp_dropout1 = hp.Float('drop_out1', min_value = 0.0, max_value = 0.5, step = 0.1)
  hp_dropout2 = hp.Float('drop_out2', min_value = 0.0, max_value = 0.5, step = 0.1)
  model.add(keras.layers.Dense(units = hp_units1, activation = 'relu'))
  model.add(keras.layers.Dropout(hp_dropout1))
  model.add(keras.layers.Dense(units = hp_units2, activation = 'relu'))
  model.add(keras.layers.Dropout(hp_dropout2))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Khởi tạo bộ điều chỉnh (tuner) và thực hiệu hypertuning

Keras Tuner có 4 bộ điều chỉnh (tuners) - `RandomSearch`, `Hyperband`, `BayesianOptimization`, và `Sklearn`. Trong hướng dẫn này, chúng ta sử dụng bộ chỉnh [Hyperband](https://arxiv.org/pdf/1603.06560.pdf). 

Để khởi tạo bộ chỉnh Hyperband, bạn phải chỉ định hypermodel, `objective` cần tối ưu hóa và số epochs tối đa để train (`max_epochs`).

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt',
                     )

Thuật toán điều chỉnh siêu băng thông (Hyperband tuning algorithm) sử dụng phân bổ tài nguyên thích ứng (adaptive resource allocation) và dừng sớm (early-stopping) để nhanh chóng hội tụ vào một mô hình hiệu suất cao (high-performing model). Điều này được thực hiện bằng cách sử dụng phương pháp kiểu vô địch thể thao.

Thuật toán đào tạo một số lượng lớn các mô hình trong một vài epochs và chỉ chuyển tiếp một nửa số mô hình hoạt động tốt nhất sang vòng tiếp theo.

Siêu băng tần xác định số lượng mô hình cần đào tạo trong một dải đồng hạng bằng cách tính toán 1 + log<sub>`factor`</sub>(`max_epochs`) và làm tròn nó đến số nguyên gần nhất.
 

Trước khi chạy tìm kiếm siêu tham số, hãy định nghĩa một lệnh callback để xóa kết quả huấn luyện ở cuối mỗi bước huấn luyện. 

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Chạy tìm kiếm siêu tham số. Các đối số cho phương thức tìm kiếm cũng giống như các đối số được sử dụng cho `tf.keras.model.fit` ngoài lệnh callback ở trên. 

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units1')}, the second fc is {best_hps.get('units2')}, the first dropout is {best_hps.get('drop_out1')}
the second dropout is {best_hps.get('drop_out2')}
and best learning rate is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 40s]
val_accuracy: 0.8822000026702881

Best val_accuracy So Far: 0.8865000009536743
Total elapsed time: 00h 08m 09s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480, the second fc is 480, the first dropout is 0.30000000000000004
the second dropout is 0.4
and best learning rate is 0.0001.



Để kết thúc hướng dẫn này, hãy đào tạo lại mô hình với các siêu tham số tối ưu vừa tìm kiếm.

In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9194 - accuracy: 0.6856 - val_loss: 0.4579 - val_accuracy: 0.8397
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4658 - accuracy: 0.8333 - val_loss: 0.4162 - val_accuracy: 0.8493
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4098 - accuracy: 0.8550 - val_loss: 0.3838 - val_accuracy: 0.8610
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3698 - accuracy: 0.8655 - val_loss: 0.3638 - val_accuracy: 0.8681
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3458 - accuracy: 0.8735 - val_loss: 0.3571 - val_accuracy: 0.8704
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3348 - accuracy: 0.8778 - val_loss: 0.3438 - val_accuracy: 0.8777
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3112 - accuracy: 0.8880 - val_loss: 0.3397 - val_accuracy:

Thư mục `my_dir/intro_to_kt` chứa nhật ký chi tiết và các điểm kiểm tra (checkpoints) cho mỗi lần chạy thử (cấu hình mô hình) chạy trong quá trình tìm kiếm siêu tham số (hyperparameter search).  Nếu bạn chạy lại tìm kiếm siêu tham số, Keras Tuner sử dụng trạng thái hiện có từ các nhật ký này để tiếp tục tìm kiếm. Để vô hiệu hóa hành vi này, hãy truyền một đối số `overwrite = True` trong khi khởi tạo bộ chỉnh (tuner).

## Tóm lược

Trong hướng dẫn này, bạn đã học cách sử dụng Keras Tuner để điều chỉnh các siêu tham số cho một mô hình. Để tìm hiểu thêm về Keras Tuner, hãy xem các tài nguyên bổ sung sau:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Ngoài ra, hãy xem  [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) trong TensorBoard để điều chỉnh tương tác các hyperparameters của mô hình 